# Libraries

In [1]:
!python -m pip install scikit-learn==1.3.1

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import TargetEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler, OneHotEncoder, power_transform, PowerTransformer
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import yeojohnson
import pickle
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
import dill
import os
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Input
from define_function import *

# Load Data

In [3]:
df = load_data('train.csv')

In [4]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


# Drop Features

In [5]:
df = drop_features(df, features_to_drop=['Alley','PoolQC','Fence','MiscFeature', ])

# Clean Data

In [6]:
df = clean_data(df, target='SalePrice')

In [7]:
df.isnull().sum().sum()

0

# Encode Data

In [8]:
Target_Encoding_list = ['MSZoning', 'Street', 'Utilities', 'LotConfig', 'Neighborhood', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'SaleType']
Ordinal_Encoding_list= ['LotShape', 'LandContour', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleCondition']

encoding_methods = {col: 'target' for col in Target_Encoding_list}
encoding_methods.update({col: 'ordinal' for col in Ordinal_Encoding_list}) # merge the dict with the target dict

df = encode_data(df, encoding_methods, train=True, target =['SalePrice'])


c:\Users\elige\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
# check if al categorical were encoded
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols

[]

In [10]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,190995.949254,65.0,8450.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,173406.590852,4.0,208500
1,2.0,20.0,190995.949254,80.0,9600.0,181130.394622,3.0,3.0,180950.936426,177972.952552,...,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,173406.590852,4.0,181500
2,3.0,60.0,190995.949254,68.0,11250.0,181130.394622,0.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,173406.590852,4.0,223500
3,4.0,70.0,190995.949254,60.0,9550.0,181130.394622,0.0,3.0,180950.936426,181620.429673,...,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,173406.590852,0.0,140000
4,5.0,60.0,190995.949254,84.0,14260.0,181130.394622,0.0,3.0,180950.936426,177972.952552,...,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,173406.590852,4.0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,190995.949254,62.0,7917.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,173406.590852,4.0,175000
1456,1457.0,20.0,190995.949254,85.0,13175.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,173406.590852,4.0,210000
1457,1458.0,70.0,190995.949254,66.0,9042.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,173406.590852,4.0,266500
1458,1459.0,20.0,190995.949254,68.0,9717.0,181130.394622,3.0,3.0,180950.936426,176941.547778,...,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,173406.590852,4.0,142125


# Transform Target

In [11]:
df = transform_data(df, 'SalePrice')

In [12]:
df.transform_target

0       0.571154
1       0.227627
2       0.741868
3      -0.425386
4       1.015293
          ...   
1455    0.136679
1456    0.588811
1457    1.170202
1458   -0.387136
1459   -0.293072
Name: transform_target, Length: 1460, dtype: float64

# Train_Test_Split

In [13]:
X_train, X_test, y_train, y_test = split_data(df, target ='transform_target', col_dropped = ['SalePrice','Id'], feature_selected= None)

# Train Model

In [14]:
model_lr = train_model(LinearRegression, xtrain=X_train, ytrain=y_train)

with open('trained_model_LR.pickle', 'wb') as f:
    dill.dump(model_lr, f)

In [15]:
model_lr

LinearRegression()

In [16]:
param_grid_xg = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [50, 100, 200, 300],
    'gamma': [0, 0.1, 0.5],
    'subsample': [0.5, 0.8, 1]
}


model_XG = train_model(XGBRegressor, xtrain=X_train, ytrain=y_train, param_grid=param_grid_xg, best_combination=True)

In [17]:
with open('trained_model_XG.pickle', 'wb') as f:
    dill.dump(model_XG, f)

In [18]:
param_grid_rf = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}


model_rf = train_model(RandomForestRegressor, xtrain=X_train, ytrain=y_train, param_grid=param_grid_rf, best_combination=True)

c:\Users\elige\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
1440 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
972 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\elige\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\elige\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\elige\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ba

In [19]:
with open('trained_model_rf.pickle', 'wb') as f:
    dill.dump(model_rf, f)

In [20]:
with open('feature_list.pickle', 'wb') as f:
    dill.dump(X_train.columns.tolist(), f)

# Build Neural Network Model

In [21]:
# Split data without transformation
X_train_nn, X_test_nn, y_train_nn, y_test_nn = split_data(df, target ='SalePrice', col_dropped = ['transform_target','Id'], feature_selected= None)

In [22]:
X_train_nn.shape

(1168, 75)

In [23]:
y_train_nn.shape

(1168,)

In [24]:
model_nn, history = neural_network_model(X=X_train_nn, y=y_train_nn, loss='mse', metrics='mse', activations='relu', widths=[64], num_layers=2, epochs=100, learning_rate=0.0001, validation_split = 0.3333)


with open('trained_nn_model.pickle', 'wb') as f:
    dill.dump(model_nn, f)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 13646707712.0000 - mse: 13646707712.0000 - val_loss: 7254137344.0000 - val_mse: 7254137344.0000
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6631764480.0000 - mse: 6631764480.0000 - val_loss: 4200463104.0000 - val_mse: 4200463104.0000
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4317946368.0000 - mse: 4317946368.0000 - val_loss: 3559481344.0000 - val_mse: 3559481344.0000
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4635144192.0000 - mse: 4635144192.0000 - val_loss: 3461703936.0000 - val_mse: 3461703936.0000
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4527638016.0000 - mse: 4527638016.0000 - val_loss: 3349314048.0000 - val_mse: 3349314048.0000
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4370728448.0000 - mse: 4370728448.0000 - val_loss: 3213877760.0000 - val_mse: 3213877760.0000
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3403463680.0000 - mse

In [25]:
model_nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,269 (106.52 KB)

 Trainable params: 9,089 (35.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,180 (71.02 KB)